In [1]:
import pandas as pd
import joblib
import json
#model libraries
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
#Result libraries
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import metrics

# TTS

In [5]:
# Function 1: Train/Test Split Evaluation

def evaluate_models_tts(datasets, models, target="rainfall", test_size=0.2, random_state=42):
    results = []
    for df_name, df in datasets.items():
        X = df.drop(columns=[target])
        y = df[target]

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state, stratify=y
        )

        for model_name, model in models.items():
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            results.append({
                "Dataset": df_name,
                "Model": model_name,
                "Accuracy": accuracy_score(y_test, y_pred),
                "Precision": precision_score(y_test, y_pred, zero_division=0),
                "Recall": recall_score(y_test, y_pred, zero_division=0),
                "F1-score": f1_score(y_test, y_pred, zero_division=0),
            })

    return pd.DataFrame(results)


In [4]:
#df1 =pd.read_csv("C:/Users/Sultan/Downloads/ML/weather forecasting/1_prep_rain.csv")
df2 = pd.read_csv('D:/Machine Learning Projects/3. Rain Predictor/2_syn_rain_1.csv')
#df3 = pd.read_csv("C:/Users/Sultan/Downloads/ML/weather forecasting/3_prep_sel_rain.csv")
#df4 = pd.read_csv("C:/Users/Sultan/Downloads/ML/weather forecasting/4_syn_sel.csv")

In [3]:
df1=df1.drop(['pop'],axis=1)
df2=df2.drop(['pop'],axis=1)
df3=df3.drop(['pop'],axis=1)

In [4]:
df4=df4.drop(['pop'],axis=1)

In [6]:
# Load your datasets
datasets = {
    "Preprocessed":df1,
    "Synthesized": df2,
    "Preprocessed_Selected": df3,
    "Synthesized_Selected": df4,
}

# Define models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
    "XGBoost": xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42),
}

# Evaluate with train/test split
tts_results = evaluate_models_tts(datasets, models)
print("\n--- Train/Test Split Results ---")
print(tts_results)

E:\anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:43:53] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
E:\anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:43:54] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
E:\anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:43:55] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
E:\anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:43:56] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



--- Train/Test Split Results ---
                  Dataset                Model  Accuracy  Precision    Recall  \
0            Preprocessed  Logistic Regression  0.951389   1.000000  0.416667   
1            Preprocessed        Random Forest  1.000000   1.000000  1.000000   
2            Preprocessed              XGBoost  1.000000   1.000000  1.000000   
3             Synthesized  Logistic Regression  0.996212   0.992481  1.000000   
4             Synthesized        Random Forest  1.000000   1.000000  1.000000   
5             Synthesized              XGBoost  1.000000   1.000000  1.000000   
6   Preprocessed_Selected  Logistic Regression  0.930556   0.666667  0.333333   
7   Preprocessed_Selected        Random Forest  0.909722   0.400000  0.166667   
8   Preprocessed_Selected              XGBoost  0.895833   0.333333  0.250000   
9    Synthesized_Selected  Logistic Regression  0.852273   0.860465  0.840909   
10   Synthesized_Selected        Random Forest  0.965909   0.955556  0.9772

In [7]:
tts_results

,Dataset,Model,Accuracy,Precision,Recall,F1-score
0,Preprocessed,Logistic Regression,0.951389,1.000000,0.416667,0.588235
1,Preprocessed,Random Forest,1.000000,1.000000,1.000000,1.000000
2,Preprocessed,XGBoost,1.000000,1.000000,1.000000,1.000000
3,Synthesized,Logistic Regression,0.996212,0.992481,1.000000,0.996226
4,Synthesized,Random Forest,1.000000,1.000000,1.000000,1.000000
5,Synthesized,XGBoost,1.000000,1.000000,1.000000,1.000000
6,Preprocessed_Selected,Logistic Regression,0.930556,0.666667,0.333333,0.444444
7,Preprocessed_Selected,Random Forest,0.909722,0.400000,0.166667,0.235294
8,Preprocessed_Selected,XGBoost,0.895833,0.333333,0.250000,0.285714
9,Synthesized_Selected,Logistic Regression,0.852273,0.860465,0.840909,0.850575


# k-Fold

In [12]:
# Function 2: K-Fold Cross Validation Evaluation
def evaluate_models_kfold(datasets, models, target="rainfall", n_splits=5, random_state=42):
    results = []

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    for df_name, df in datasets.items():
        X = df.drop(columns=[target])
        y = df[target]

        for model_name, model in models.items():
            scores = cross_validate(
                model, X, y, cv=skf,
                scoring=["accuracy", "precision", "recall", "f1"],
                return_train_score=False
            )

            results.append({
                "Dataset": df_name,
                "Model": model_name,
                "Accuracy": scores["test_accuracy"].mean(),
                "Precision": scores["test_precision"].mean(),
                "Recall": scores["test_recall"].mean(),
                "F1-score": scores["test_f1"].mean(),
            })

    return pd.DataFrame(results)

In [13]:
# Evaluate with K-Fold
kfold_results = evaluate_models_kfold(datasets, models)
print("\n--- K-Fold Results ---")
print(kfold_results)

E:\anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:46:03] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
E:\anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:46:03] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
E:\anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:46:04] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
E:\anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:46:04] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
E:\anaconda\Lib\site-pac


--- K-Fold Results ---
                  Dataset                Model  Accuracy  Precision    Recall  \
0            Preprocessed  Logistic Regression  0.968056   1.000000  0.616667   
1            Preprocessed        Random Forest  1.000000   1.000000  1.000000   
2            Preprocessed              XGBoost  1.000000   1.000000  1.000000   
3             Synthesized  Logistic Regression  0.987879   0.990851  0.984848   
4             Synthesized        Random Forest  0.999242   1.000000  0.998485   
5             Synthesized              XGBoost  1.000000   1.000000  1.000000   
6   Preprocessed_Selected  Logistic Regression  0.926389   0.640952  0.283333   
7   Preprocessed_Selected        Random Forest  0.925000   0.640000  0.233333   
8   Preprocessed_Selected              XGBoost  0.925000   0.633333  0.316667   
9    Synthesized_Selected  Logistic Regression  0.856818   0.854499  0.860606   
10   Synthesized_Selected        Random Forest  0.971212   0.962808  0.980303   
11  

In [14]:
kfold_results

,Dataset,Model,Accuracy,Precision,Recall,F1-score
0,Preprocessed,Logistic Regression,0.968056,1.000000,0.616667,0.760702
1,Preprocessed,Random Forest,1.000000,1.000000,1.000000,1.000000
2,Preprocessed,XGBoost,1.000000,1.000000,1.000000,1.000000
3,Synthesized,Logistic Regression,0.987879,0.990851,0.984848,0.987815
4,Synthesized,Random Forest,0.999242,1.000000,0.998485,0.999240
5,Synthesized,XGBoost,1.000000,1.000000,1.000000,1.000000
6,Preprocessed_Selected,Logistic Regression,0.926389,0.640952,0.283333,0.387217
7,Preprocessed_Selected,Random Forest,0.925000,0.640000,0.233333,0.336732
8,Preprocessed_Selected,XGBoost,0.925000,0.633333,0.316667,0.401861
9,Synthesized_Selected,Logistic Regression,0.856818,0.854499,0.860606,0.857235


# SAVING MODEL

In [5]:
X = df2.drop(columns=["rainfall"])     # features
y = df2["rainfall"]                    # target

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)


In [10]:
print(X.columns.tolist())


['slp', 'dewpt', 'uv', 'wind_dir', 'precip', 'pop', 'ozone', 'app_temp', 'clouds_low', 'clouds_mid', 'wind_gust_spd', 'dni', 'rh', 'pod', 'pres', 'temp', 'clouds', 'vis', 'clouds_hi', 'wind_spd', 'lon', 'lat', 'month_sin', 'month_cos', 'hour_sin', 'hour_cos', 'wind_deg', 'wind_sin', 'wind_cos', 'city_Bahāwalpur', 'city_Faisalābād', 'city_Gujrānwāla', 'city_Hyderabad', 'city_Islamabad', 'city_Karachi', 'city_Lahore', 'city_Mandi Bahāuddīn', 'city_Miānwāli', 'city_Multān', 'city_Peshawar', 'city_Quetta', 'city_Sargodha', 'city_Sheikhupura', 'city_Sialkot', 'city_Skārdu', 'city_Turbat', 'city_Tāl', 'country_Pakistan', 'weather_category_clear', 'weather_category_clouds', 'weather_category_rain', 'weather_category_thunderstorm']


In [6]:
model_xgb = xgb.XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    use_label_encoder=False,
    eval_metric="logloss"  # prevents warning
)

In [7]:
# Train
model_xgb.fit(X_train, y_train)

# Predict
y_xgb_predict = model_xgb.predict(X_test)

D:\ANACONDA\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:58:14] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [8]:
# Evaluate
print("Accuracy: ", metrics.accuracy_score(y_test, y_xgb_predict))
print("Precision: ", metrics.precision_score(y_test, y_xgb_predict, zero_division=0))
print("Recall: ", metrics.recall_score(y_test, y_xgb_predict, zero_division=0))
print("F1_Score: ", metrics.f1_score(y_test, y_xgb_predict, zero_division=0))

Accuracy:  1.0
Precision:  1.0
Recall:  1.0
F1_Score:  1.0


In [9]:
joblib.dump(model_xgb,"D:/Machine Learning Projects/3. Rain Predictor/xgb_model_1.pkl")

['D:/Machine Learning Projects/3. Rain Predictor/xgb_model_1.pkl']

In [20]:
df2.head()

,slp,dewpt,uv,wind_dir,precip,ozone,app_temp,clouds_low,clouds_mid,wind_gust_spd,...,city_Sialkot,city_Skārdu,city_Turbat,city_Tāl,country_Pakistan,weather_category_clear,weather_category_clouds,weather_category_rain,weather_category_thunderstorm,rainfall
0,0.645640,0.265971,-0.791098,0.387508,-0.180781,-0.293695,0.295985,3.437158,-0.518372,0.036598,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0
1,0.475826,0.265971,-0.791098,0.479079,-0.180781,-0.878623,0.278430,4.792834,-0.518372,0.101065,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0
2,0.475826,0.265971,-0.791098,0.421847,0.027146,-0.460817,0.214063,3.630826,-0.518372,0.342816,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1
3,0.702245,0.265971,-0.054955,0.433293,-0.180781,-0.410680,0.366204,4.018162,-0.518372,0.326699,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0
4,0.645640,0.265971,1.785403,0.421847,-0.180781,-0.795062,0.664635,2.662486,-0.518372,0.471750,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0


In [12]:
# Compute column means
feature_means = df2.mean(numeric_only=True).to_dict()

# Save to JSON
with open("feature_means.json", "w") as f:
    json.dump(feature_means, f, indent=4)

print("✅ feature_means.json created successfully!")

✅ feature_means.json created successfully!


## Model Deployement

In [ ]:
import os
import json
from datetime import datetime

import joblib
import numpy as np
import pandas as pd
import requests
from flask import Flask, request, jsonify, render_template
from pyngrok import ngrok

# =======================
# PATHS / CONFIG
# =======================
BASE_DIR = r"D:/Machine Learning Projects/3. Rain Predictor/"
TEMPLATE_DIR = r"D:/Machine Learning Projects/3. Rain Predictor/"
STATIC_DIR = os.path.join(TEMPLATE_DIR, "static")

OPENWEATHER_API_KEY = "958a0a19dccc5dc2c18f48d153207c33"

# ngrok
NGROK_AUTH_TOKEN = "31p88w2mFBhEjpQg9XfjhjOJc6u_7LF9KMq6ap49d8A15LWj2"
PORT = 5001

# =======================
# ARTIFACTS
# =======================
model = joblib.load(os.path.join(BASE_DIR, "xgb_model_1.pkl"))
scaler = joblib.load(os.path.join(BASE_DIR, "standard_scaler.pkl"))
one_hot_encoder = joblib.load(os.path.join(BASE_DIR, "one_hot_encoder.pkl"))
label_encoder_pod = joblib.load(os.path.join(BASE_DIR, "label_encoder.pkl"))

# feature means (for backfilling missing numeric fields from API)
with open(os.path.join(BASE_DIR, "feature_means.json"), "r") as f:
    MEAN_VALUES = json.load(f)

# EXACT feature order used for training (after OHE/LE)
FEATURES = [
    'slp', 'dewpt', 'uv', 'wind_dir', 'precip', 'pop', 'ozone', 'app_temp',
    'clouds_low', 'clouds_mid', 'wind_gust_spd', 'dni', 'rh', 'pod',
    'pres', 'temp', 'clouds', 'vis', 'clouds_hi', 'wind_spd',
    'lon', 'lat', 'month_sin', 'month_cos', 'hour_sin', 'hour_cos',
    'wind_deg', 'wind_sin', 'wind_cos',
    'city_Bahāwalpur', 'city_Faisalābād', 'city_Gujrānwāla', 'city_Hyderabad',
    'city_Islamabad', 'city_Karachi', 'city_Lahore',
    'city_Mandi Bahāuddīn', 'city_Miānwāli', 'city_Multān',
    'city_Peshawar', 'city_Quetta', 'city_Sargodha',
    'city_Sheikhupura', 'city_Sialkot', 'city_Skārdu',
    'city_Turbat', 'city_Tāl', 'country_Pakistan',
    'weather_category_clear', 'weather_category_clouds',
    'weather_category_rain', 'weather_category_thunderstorm'
]

# numeric columns (exactly what your scaler was fit on)
SCALER_NUMERIC_COLS = list(scaler.feature_names_in_)

# =======================
# FLASK APP
# =======================
app = Flask(__name__, template_folder=TEMPLATE_DIR, static_folder=STATIC_DIR)

# Setup ngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(PORT)
print(f"🌍 Ngrok Tunnel URL: {public_url}")

# =======================
# HELPERS
# =======================
def fetch_openweather(city: str) -> dict:
    """
    Fetch current weather for a PK city via OpenWeather (metric units).
    """
    url = (
        f"http://api.openweathermap.org/data/2.5/weather"
        f"?q={city},PK&appid={OPENWEATHER_API_KEY}&units=metric"
    )
    r = requests.get(url, timeout=15)
    r.raise_for_status()
    return r.json()

def to_weather_category(ow_main: str) -> str:
    """
    Map OpenWeather 'weather[0].main' to your categories used in training.
    """
    s = (ow_main or "").lower()
    if "clear" in s:
        return "clear"
    if "cloud" in s:
        return "clouds"
    if "rain" in s:
        return "rain"
    if "thunder" in s:
        return "thunderstorm"
    return "other"

def build_raw_row_from_openweather(city: str, res: dict) -> dict:
    """
    Build a single raw row (pre-encoding & pre-scaling) following your training logic.
    - Fill what OpenWeather provides.
    - Backfill the rest with MEAN_VALUES.
    - Build time/wind cyclical features exactly as you trained.
    """
    now = datetime.utcnow()
    month = now.month
    hour = now.hour

    main = res.get("main", {})
    wind = res.get("wind", {})
    clouds = res.get("clouds", {})
    coord = res.get("coord", {})
    weather_list = res.get("weather", [{}])
    ow_main = weather_list[0].get("main", "")

    # Core numeric features (fill from API where available; else mean)
    row = {
        "slp": main.get("pressure", MEAN_VALUES.get("slp", 0)),   # hPa
        "pres": main.get("pressure", MEAN_VALUES.get("pres", 0)), # same as slp in your training
        "temp": main.get("temp", MEAN_VALUES.get("temp", 0)),
        "app_temp": main.get("feels_like", MEAN_VALUES.get("app_temp", 0)),
        "rh": main.get("humidity", MEAN_VALUES.get("rh", 0)),
        "clouds": clouds.get("all", MEAN_VALUES.get("clouds", 0)),
        "vis": (res.get("visibility", MEAN_VALUES.get("vis", 0)) / 1000.0) if "visibility" in res else MEAN_VALUES.get("vis", 0),  # km
        "wind_spd": wind.get("speed", MEAN_VALUES.get("wind_spd", 0)),
        "wind_gust_spd": wind.get("gust", MEAN_VALUES.get("wind_gust_spd", 0)) if "gust" in wind else MEAN_VALUES.get("wind_gust_spd", 0),
        "wind_dir": wind.get("deg", MEAN_VALUES.get("wind_dir", 0)),
        "wind_deg": wind.get("deg", MEAN_VALUES.get("wind_deg", 0)),
        "lat": coord.get("lat", MEAN_VALUES.get("lat", 0)),
        "lon": coord.get("lon", MEAN_VALUES.get("lon", 0)),
        # fields OpenWeather doesn't provide directly → backfill from means you saved
        "dewpt": MEAN_VALUES.get("dewpt", 0),
        "uv": MEAN_VALUES.get("uv", 0),
        "precip": MEAN_VALUES.get("precip", 0),
        "pop": MEAN_VALUES.get("pop", 0),
        "ozone": MEAN_VALUES.get("ozone", 0),
        "clouds_low": MEAN_VALUES.get("clouds_low", 0),
        "clouds_mid": MEAN_VALUES.get("clouds_mid", 0),
        "clouds_hi": MEAN_VALUES.get("clouds_hi", 0),
        "dni": MEAN_VALUES.get("dni", 0),
    }

    # Time-based cyclical features (month/hour)
    row["month_sin"] = np.sin(2 * np.pi * month / 12)
    row["month_cos"] = np.cos(2 * np.pi * month / 12)
    row["hour_sin"]  = np.sin(2 * np.pi * hour / 24)
    row["hour_cos"]  = np.cos(2 * np.pi * hour / 24)

    # Wind trig transforms
    deg = row["wind_deg"]
    row["wind_sin"] = np.sin(np.radians(deg))
    row["wind_cos"] = np.cos(np.radians(deg))

    # POD (day/night) → use hour like you did
    pod_char = "d" if 6 <= hour <= 18 else "n"

    # Categorical originals (for encoders)
    row["city"] = city
    row["country"] = "Pakistan"
    row["weather_category"] = to_weather_category(ow_main)
    row["pod_raw"] = pod_char   # keep raw to encode via saved LabelEncoder

    return row

def encode_and_scale(row: dict) -> pd.DataFrame:
    """
    Apply your saved LabelEncoder for 'pod', your saved OneHotEncoder for ['city','country','weather_category'],
    then scale numeric columns with your saved StandardScaler.
    Finally, align to FEATURES order, adding any missing OHE columns as 0.
    """
    # 1) DataFrame from row
    df_raw = pd.DataFrame([row])

    # 2) Label-encode 'pod' exactly like training
    #    You saved ONE label encoder that was fit on df['pod'] (values 'd'/'n')
    df_raw["pod"] = label_encoder_pod.transform([row["pod_raw"]])[0]

    # 3) OneHot-encode ('city','country','weather_category')
    cats = df_raw[["city", "country", "weather_category"]]
    cats_ohe = one_hot_encoder.transform(cats)
    cats_ohe_cols = one_hot_encoder.get_feature_names_out()
    df_cats = pd.DataFrame(cats_ohe, columns=cats_ohe_cols, index=df_raw.index)

    # 4) Merge back numeric + engineered + LE 'pod'
    #    Drop the original raw cat columns
    df_full = df_raw.drop(columns=["city", "country", "weather_category", "pod_raw"]).join(df_cats)

    # 5) Scale numeric columns that the scaler knows
    #    Some columns (OHE) are not scaled and that's correct.
    intersect_cols = [c for c in SCALER_NUMERIC_COLS if c in df_full.columns]
    df_full.loc[:, intersect_cols] = scaler.transform(df_full[intersect_cols])

    # 6) Rebuild in exact FEATURES order (add any missing columns as 0)
    final = pd.DataFrame(columns=FEATURES)
    for col in FEATURES:
        final[col] = df_full[col] if col in df_full.columns else 0

    # Ensure numeric dtypes
    final = final.astype(float)

    return final

# =======================
# ROUTES
# =======================
@app.route("/")
def index():
    return render_template("index.html")

@app.route("/predict", methods=["POST"])
def predict():
    try:
        city = request.form.get("city", "").strip()
        if not city:
            return jsonify({"error": "City is required."}), 400

        # 1) Fetch live weather
        res = fetch_openweather(city)

        # 2) Build raw row (pre-encoding/scaling)
        row = build_raw_row_from_openweather(city, res)

        # 3) Encode + scale + align feature order
        features_df = encode_and_scale(row)

        # 4) Predict
        pred = model.predict(features_df)[0]

        return jsonify({
            "city": city,
            "prediction": int(pred),
            "features_used": features_df.to_dict(orient="records")[0]
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# =======================
# MAIN
# =======================
if __name__ == "__main__":
    app.run(port=PORT)


🌍 Ngrok Tunnel URL: NgrokTunnel: "https://0aebda7bcd22.ngrok-free.app" -> "http://localhost:5001"
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [30/Aug/2025 16:17:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Aug/2025 16:17:55] "GET /favicon.ico HTTP/1.1" 404 -
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2020\523977647.py:104: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow()
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2020\523977647.py:190: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.87205898]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_full.loc[:, intersect_cols] = scaler.transform(df_full[intersect_cols])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2020\523977647.py:190: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise i